# 웹캠으로 CCTV 만들기

* 난이도 : ★★★★☆☆☆☆☆☆
* 필요라이브러리: numpy, openCV, scikit-image, pillow


* 웹캠을 사용하여 화면을 감시하다 화면에 변화가 생기면 인지하는 기능의 프로그램을 만듭니다.
* scikit-image 는 기하학변형, 색공간 조작, 분석, 필터링, 형상감지 등의 알고리즘이 포함된 이미지 처리를 위한 라이브러리 입니다.
> pip install scikit-image

## 필요라이브러리 import

In [ ]:
# openCV 를 사용하기 위한 라이브러리
import cv2

# PIL 이미지와 openCV 이미지를 서로 변환하기 위해 사용
import numpy as np

# 이미지의 유사도를 측정하기 위한 라이브러리
from skimage.measure import compare_ssim

# 웹캠 화면에 유사도를 글자로 출력하기 위해 텍스트를 그리기 위해 사용
from PIL import ImageFont, ImageDraw, Image

## 기본 로직

In [ ]:
# 웹캠 캡쳐 오픈
cap = cv2.VideoCapture(0)
 
# 웹캠 사용가능한지 체크
if (cap.isOpened() == False): 
    print("카메라를 오픈할 수 없습니다.")
 
# 연결된 웹캠 디바이스의 프레임 해상도를 알아옵니다.
# float 를 int 로 캐스팅 해야 합니다.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# 무한루프
while(True):
    # 웹캠 장치에서 프레임을 읽어옵니다.
    ret, frame = cap.read()
 
    # 프레임 읽기에 성공하면
    if ret == True: 
        # 이미지 출력
        cv2.imshow('CCTV',frame)

        # Q 키 누르면 종료합니다.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # 프레임 읽기 실패시 종료
    else:
        break 
 
# 웹캠 장치와 파일저장을 모두 종료시킵니다.
cap.release()
 
# opencv 로 생성된 창을 모두 삭제 합니다.
cv2.destroyAllWindows() 


## 기능 추가
* 디버깅을 위해 웹캠 내용을 파일로 저장하는 기능을 추가 합니다.
* 이미지의 유사도를 계산하여 화면의 변화가 있는지를 감지하는 기능을 추가합니다.
* 이미지 유사도를 직접 이미지에 작성하는 기능을 추가합니다.

In [ ]:
import cv2
import numpy as np
from skimage.measure import compare_ssim
from PIL import ImageFont, ImageDraw, Image

def make_text(img, text):
    '''이미지 우하단에 텍스트를 작성하는 함수
    opencv 에서 제공하는 텍스트는 기능이 너무 제한적이라 pillow 로 텍스트를 작성합니다.
    Args:
        img (openc_cv) : opencv 이미지
        text (str) : 텍스트
    
    Returns:
        opencv : 텍스트가 작성된
    '''

    # 폰트명과 사이즈를 인자로 넘겨줘 PIL 의 ImageFont 객체 생성
    font = ImageFont.truetype("malgun.ttf", 17)
    
    # 해당 텍스트의 영역을 구합니다.
    text_w, text_h = font.getsize(text)

    # 이미지의 w, h 
    w = img.shape[1]
    h = img.shape[0]

    # 텍스트가 실제 이미지에 위치할 위치값 계산
    X_POS = w - text_w - 10
    Y_POS = h - text_h - 10
    
    # 이미지의 텍스트 영역에 투명 사각형을 그리기 위해서는 오버레이 처리를 해야합니다.
    # 먼저 원본 이미지의 복사본을 생성
    overlay = img.copy()
    
    # 복사본 이미지의 텍스트 영역에 사각형을 Fill 형태로 채웁니다.
    cv2.rectangle(overlay, (X_POS - 3, Y_POS), (X_POS + text_w, Y_POS + text_h + 3), (0, 0, 0), -1)

    # 사각형의 투명도를 설정합니다.
    alpha = 0.5 
    
    # 원본이미지와 복사본 이미지를 합칩니다.
    # addWeighted 함수는 이미지를 합칠시에 가중치를 주어 합치는 방식인데
    # 여기서는 alpha 값만큼을 1-alpha 값만큼으로 합치게 되니 실제 투명박스 부분만 합쳐지게 됩니다. 
    img = cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0)

    # opencv -> pillow
    img_pil = Image.fromarray(img)

    # pillow 이미지로 draw 객체는 생성합니다.
    draw = ImageDraw.Draw(img_pil)
    
    # 이미지에 텍스트를 작성합니다.
    draw.text((X_POS, Y_POS), text, (255,255,255), font=font)

    # pillow -> opencv    
    img = np.array(img_pil)

    return img


# 웹캠 캡쳐 오픈
cap = cv2.VideoCapture(0)
 
# 웹캠 사용가능한지 체크
if (cap.isOpened() == False): 
    print("카메라를 오픈할 수 없습니다.")
 
# 연결된 웹캠 디바이스의 프레임 해상도를 알아옵니다.
# float 를 int 로 캐스팅 해야 합니다.
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
 
# 디버깅을 위해서 웹캠의 내용을 파일로 저장하기 위해 VideoWriter 를 설정합니다.
# 여기서 'M', 'J', 'P', 'G' 는 MotionJPEG 코덱을 사용한다는 이야기인데
# D I V X 같은 코덱도 해당 컴퓨터에 설치 되어있으면 사용할 수 있습니다.
# 10은 FPS 로 10프레임으로 저장을 한다는 이야기 입니다.
out = cv2.VideoWriter('outpy.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

# 이미지 변화를 감지하기 위해 이전 이미지를 저장해놓을 변수
old_image = None

# 화면 변화 감지시 빨간색 박스를 그리기 위해 (BGR)
c = (0, 0, 255)

# 무한루프
while(True):
    # 웹캠 장치에서 프레임을 읽어옵니다.
    ret, frame = cap.read()
 
    # 프레임 읽기에 성공하면
    if ret == True: 
        # 이전 이미지가 존재한다면
        if old_image is not None:
            # 현재 이미지와 이전 이미지를 흑백으로 변환합니다.
            grayA = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            grayB = cv2.cvtColor(old_image, cv2.COLOR_BGR2GRAY)

            # 사이킷 이미지의 compare_ssim 함수를 사용하여
            # 이미지의 유사도를 측정합니다.
            (score, diff) = compare_ssim(grayA, grayB, full=True)
            diff = (diff * 255).astype("uint8")
            print("SSIM: {}".format(score))


            # 현재 프레임의 width, height 을 구합니다.
            w = frame.shape[1]
            h = frame.shape[0]

            # 이미지에 유사도를 출력
            frame = make_text(frame, "유사도: {:.12f}".format(score))

            # 이미지 유사도가 xxx이하면
            if score < 0.90:
                # 이미지에 6 두께의 빨간 사각형을 그립니다.
                cv2.rectangle(frame, (0, 0), (w, h), c, 6)

        # 다음 프레임 비교를 위해 현재 프레임을 이전 이미지 변수에 저장
        old_image = frame

        # 이미지 출력
        cv2.imshow('CCTV',frame)

        # 파일로 저장
        out.write(frame)

        # Q 키 누르면 종료합니다.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    # 프레임 읽기 실패시 종료
    else:
        break 
 
# 웹캠 장치와 파일저장을 모두 종료시킵니다.
cap.release()
out.release()
 
# opencv 로 생성된 창을 모두 삭제 합니다.
cv2.destroyAllWindows() 
